In [4]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))



df = pd.read_csv(os.path.join(os.getcwd(), "../../data/total_4am.csv"), index_col=0)


In [5]:
from performance import get_performance

def get_results(k, df, ma):

    original_df = df.copy()  # Make a copy of the original DataFrame
    df["range"] = df["high"] - df["low"]

    for i in range(1, len(df)):
        df.loc[i, "target_price"] = df.loc[i-1, "range"] * k + df.loc[i, "open"]

    df[f"ma_{ma}"] = df["open"].rolling(window=ma).mean()


    df['signal'] = 0  # Default to no position
    # Calculate the strategy returns (only when in a long position)
    df['strategy_returns'] = 0.0
    df['strategy_returns2'] = 0.0


    for i in range(ma, len(df)):
        # Buy condition
        if df.loc[i, "high"] >= df.loc[i, "target_price"] and df.loc[i, "open"] >= df.loc[i, f"ma_{ma}"]:
            df.loc[i, 'signal'] = 1
            df.loc[i, "strategy_returns"] = df.loc[i, "close"]/df.loc[i, "target_price"] - 1
            df.loc[i, "strategy_returns2"] = (df.loc[i, "close"] * 0.998)/(df.loc[i, "target_price"] * 1.002) - 1


    df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

    df["cumulative_returns2"] = (1 + df['strategy_returns2']).cumprod()


    # Calculate the benchmark cumulative returns (buy and hold strategy)
    df['benchmark_returns'] = (1 + df['close'].pct_change()).cumprod()

    # Save the results to a CSV file
    df.to_csv(f"../../results/volatilitybreakout/volatility_breakout_basic_{k}_{ma}.csv")

    # Get performance metrics and plot the results
    get_performance(df=df, title=f"Volatility Breakout_{k}_{ma}")

    df = original_df.copy()  # Create a fresh copy for each interval


In [6]:
ks = [k / 10.0 for k in range(1, 10)]
mas = [5, 10, 20, 50, 120, 200]
for k in ks:
    for ma in mas:
        get_results(k=k, df=df, ma=ma)

======Investment Summary======
Strategy        : Volatility Breakout_0.1_5
total_return    : 187.37
cagr            : 36.75
mdd             : 33.4
total_return_w_fee : -66.46
cagr_w_fee      : -27.67
mdd_w_fee       : 71.21
investing_days  : 1231
======Investment Summary======
Strategy        : Volatility Breakout_0.1_10
total_return    : 144.38
cagr            : 30.45
mdd             : 27.18
total_return_w_fee : -70.55
cagr_w_fee      : -30.49
mdd_w_fee       : 71.49
investing_days  : 1227
======Investment Summary======
Strategy        : Volatility Breakout_0.1_20
total_return    : 93.31
cagr            : 22.14
mdd             : 37.83
total_return_w_fee : -76.52
cagr_w_fee      : -35.57
mdd_w_fee       : 76.79
investing_days  : 1203
======Investment Summary======
Strategy        : Volatility Breakout_0.1_50
total_return    : 133.24
cagr            : 31.79
mdd             : 41.5
total_return_w_fee : -70.74
cagr_w_fee      : -33.0
mdd_w_fee       : 76.27
investing_days  : 1120
======Inv